In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy 

# Delving Deeper into Conolutional Neural Networks

In [3]:
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(20, 3, padding = "same", activation = tf.keras.activations.relu),
    Conv2D(20, 3, padding = "same", activation = tf.keras.activations.relu),
    MaxPool2D(),
    Conv2D(50, 3, padding = "same", activation = tf.keras.activations.relu),
    Conv2D(50, 3, padding = "same", activation = tf.keras.activations.relu),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(100, 3, padding = "same", activation = tf.keras.activations.relu),
    Conv2D(100, 3, padding = "same", activation = tf.keras.activations.relu),
    MaxPool2D(),
    Flatten(),
    Dense(units=20, activation=tf.keras.activations.relu),
    Dense(units=1, activation=tf.keras.activations.sigmoid)
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 20)      560       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 20)      3620      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 50)      9050      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 50)      22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 50)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 50)        2

In [5]:
model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = SparseCategoricalCrossentropy(),
    metrics = [SparseCategoricalAccuracy()]
)

In [6]:
# layer.input_shape
# layer.output_shape

In [7]:
filenames = []

# Download the set from Kaggle
# https://www.kaggle.com/c/dogs-vs-cats/data

# BASE_DIR = "./dogs-vs-cats/train"
BASE_DIR = os.path.abspath("./dogs-vs-cats/train")

# os.listdir(BASE_DIR)
# BASE_DIR + "/cat."
for image_class in os.listdir(BASE_DIR):
    folder_name = BASE_DIR + "/" + image_class
    filenames.append(image_class)
    
# len(filenames)
# [f for f in filenames if 'dog' in f]
# [f for f in filenames if 'cat' in f]

In [8]:
filenames = np.array(filenames)
classes = [f[:3] for f in filenames]

In [9]:
def read_data(filename, label):
    print(filename, label)
    
    result_file = tf.io.read_file(BASE_DIR + filename)
    result_image = tf.image.decode_image(result_file)
#     result_image = tf.image.resize(result_image, (224, 224))
    result_image = tf.cast(result_image, tf.float32) / 255.0
    return (result_image, label)

In [10]:
# This is required to process large quantity of images!
# See from_tensors too
# Use the same approach for prediction dataset

dataset = tf.data.Dataset.from_tensor_slices((filenames, classes))

# for item in dataset:
#     print(item)
#     break

# This has to be the first! ; Consider "interleave" method
dataset = dataset.map(read_data)

# Count of elements to be shuffled; consider "reshuffle_each_iteration"
dataset = dataset.shuffle(len(filenames))

# 32 images at once; should be power of 2
dataset = dataset.batch(32)

# To work for multiple epochs; No parameter - infinite repetitions
dataset = dataset.repeat()

Tensor("args_0:0", shape=(), dtype=string) Tensor("args_1:0", shape=(), dtype=string)


In [11]:
# Incomplete, Does not work now!
# model.fit(dataset, steps_per_epoch=int(len(dataset)/32) ) # same as batchsize above